In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
!git clone https://github.com/Tony607/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning


from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

conv_base = Net(weights = "imagenet", include_top = False)

from tensorflow.keras import models
from tensorflow.keras import layers



dropout_rate = 0.2 
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
model.add(layers.Flatten(name = "flatten"))
model.add(layers.Dense(196, activation = 'softmax', name = "fc_out"))
model.load_weights('/content/gdrive/My Drive/MLDS/model_weights_2.h5')
model.summary()



Cloning into 'efficientnet_keras_transfer_learning'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 171 (delta 98), reused 161 (delta 93), pack-reused 0
Receiving objects: 100% (171/171), 5.44 MiB | 7.69 MiB/s, done.
Resolving deltas: 100% (98/98), done.
/content/efficientnet_keras_transfer_learning


Using TensorFlow backend.



Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
16719872/16717576 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 7, 7, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 196)               251076    
Total params: 4,300,640
Trainable params: 4,258,624
Non-trainable params: 42,016
_________________________________________________________________


In [ ]:
batch_size = 64
epochs = 50

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.metrics import top_k_categorical_accuracy


train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                   rotation_range = 10,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


val_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/MLDS/x_train_jpeg',
                                                    target_size = (224, 224),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical")


validation_generator = val_datagen.flow_from_directory(
    '/content/gdrive/My Drive/MLDS/x_val_jpeg',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k = 5)


model.compile(loss = "categorical_crossentropy",
              optimizer = optimizers.Adam(lr = 1e-5),
              metrics = ['acc', top_5_categorical_accuracy])
          

hist =model.fit_generator(train_generator, steps_per_epoch = 8144 // batch_size, 
                          epochs = epochs,validation_data = validation_generator,
)



train_acc = hist.history['acc']
val_acc = hist.history['val_acc']

train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

print(train_acc)
print(val_acc)
print(train_loss)
print(val_loss)


epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label="Val acc")
plt.title('Training anc Val accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Val loss')
plt.title('Training and Val loss')
plt.legend()

plt.show()




Found 6000 images belonging to 196 classes.
Found 2144 images belonging to 196 classes.
Epoch 1/50
127/127 [==============================] - 204s 2s/step - loss: 0.0623 - acc: 0.9874 - top_5_categorical_accuracy: 0.9998 - val_loss: 1.7020 - val_acc: 0.6301 - val_top_5_categorical_accuracy: 0.8517
Epoch 2/50
127/127 [==============================] - 180s 1s/step - loss: 0.0653 - acc: 0.9843 - top_5_categorical_accuracy: 0.9993 - val_loss: 1.6891 - val_acc: 0.6334 - val_top_5_categorical_accuracy: 0.8545
Epoch 3/50
127/127 [==============================] - 176s 1s/step - loss: 0.0634 - acc: 0.9868 - top_5_categorical_accuracy: 0.9991 - val_loss: 1.6879 - val_acc: 0.6343 - val_top_5_categorical_accuracy: 0.8540
Epoch 4/50
127/127 [==============================] - 176s 1s/step - loss: 0.0635 - acc: 0.9847 - top_5_categorical_accuracy: 0.9995 - val_loss: 1.6886 - val_acc: 0.6367 - val_top_5_categorical_accuracy: 0.8531
Epoch 5/50
127/127 [==============================] - 177s 1s/step -

NameError: ignored

In [ ]:
# 모델 저장
from keras.models import load_model

model.save('/content/gdrive/My Drive/MLDS/MLDS_CNN_CAR196_6.h5')


model_json = model.to_json()
with open("/content/gdrive/My Drive/MLDS/model_6.json", "w") as json_file : 
    json_file.write(model_json)


model.save_weights("/content/gdrive/My Drive/MLDS/model_weights_6.h5")
print("Saved model to disk")

Saved model to disk
